In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
import tensorflow_datasets as tfds
(train_ds, val_ds), ds_info = tfds.load('tf_flowers', split=['train[:80%]', 'train[80%:]'], with_info=True, as_supervised=True)

c:\Users\HP\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 30.69 url/s]


Dataset tf_flowers downloaded and prepared to C:\Users\HP\tensorflow_datasets\tf_flowers\3.0.1. Subsequent calls will reuse this data.


In [3]:
IMG_SIZE = 160
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))    #resize all images to 160X160
    image = image / 255.0
    return image, label

In [4]:
train_ds = train_ds.map(preprocess).batch(32).shuffle(1000)
val_ds = val_ds.map(preprocess).batch(32)

In [5]:
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [6]:
# This is pretrained model MObileNetV2( with all layers frozen)
# It extracts meaninful features from images.

x = base_model.output
# This Flattern the output from the the CNN to prepare it for classification
x = GlobalAveragePooling2D()(x)
x= Dense(128, activation='relu')(x)  
# This is the classifier layer
# It takes the Features and classifies the image(eg."daisy" vs "not daisy")
predictions = Dense(5, activation='softmax')(x)

In [7]:
model= Model(inputs=base_model.input, outputs=predictions)

In [8]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(train_ds, epochs=5, validation_data=val_ds)

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 54s 485ms/step - accuracy: 0.7946 - loss: 0.5643 - val_accuracy: 0.8624 - val_loss: 0.3563
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 44s 460ms/step - accuracy: 0.9166 - loss: 0.2344 - val_accuracy: 0.8869 - val_loss: 0.3009
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 399ms/step - accuracy: 0.9550 - loss: 0.1503 - val_accuracy: 0.8951 - val_loss: 0.2952
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 36s 391ms/step - accuracy: 0.9758 - loss: 0.0936 - val_accuracy: 0.8856 - val_loss: 0.3575
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 36s 391ms/step - accuracy: 0.9874 - loss: 0.0578 - val_accuracy: 0.8978 - val_loss: 0.3358
